# Reviewing and Approving Code in Syft as a Data Owner

### Import packages

In [ ]:
SYFT_VERSION = ">=0.9,<1.0.0"
package_string = f'"syft{SYFT_VERSION}"'
# %pip install {package_string} -q

In [ ]:
# syft absolute
import syft as sy

sy.requires(SYFT_VERSION)

### Login to Syft Datasite Server

In [ ]:
# Launch and connect to test-datasite-1 server we setup in the previous notebook
server = sy.orchestra.launch(name="test-datasite-1", port="auto", dev_mode=True)

In [ ]:
# Log into the server with default root credentials
datasite_client = server.login(email="info@openmined.org", password="changethis")

### Selecting Project in the Syft Datasite Server

Let's see all the projects that are created by Data Scientists in this Datasite Server

In [ ]:
datasite_client.projects

In [ ]:
# Select the project you want to work with
project = datasite_client.projects[0]
project

All code requests submitted by the Data Scientists as a part of this project can be accessed by invoking the following

In [ ]:
project.requests

In [ ]:
# Tests
assert len(project.events) == 1
assert isinstance(project.events[0], sy.service.project.project.ProjectRequest)
assert len(project.requests) == 1

### Reviewing Code Requests

To review a specific request, we can select it and explore its attributes

In [ ]:
request = project.requests[0]
request

In [ ]:
# See the code written by the Data Scientist and its metadata in the request
func = request.code
func

In [ ]:
# To see just the code
func.show_code

In [ ]:
# Reference to the assets that the function will run on
func.assets

Viewing the Asset and it's mock/private variants that the Data Scientist will be running on

In [ ]:
asset = func.assets[0]
asset

In [ ]:
mock_data = asset.mock
mock_data

In [ ]:
# Private data. Accessible as we are logged in with Data Owner credentials
pvt_data = asset.data
pvt_data

In [ ]:
# Tests
assert len(asset.data_subjects) == 1
assert mock_data.shape == (10, 22)
assert pvt_data.shape == (10, 22)

#### Policies in Syft Function

Each Syft Function requires an Input & Output policy attached to the python function against which executions are verified.

Syft provides the following default policies:
* `sy.ExactMatch()` Input policy ensures that function executes against the exact inputs specified by Data Scientist.
* `sy.OutputPolicyExecuteOnce()` Output policy makes sure that the Data Scientist can run the function only once against the input.

We can also implement custom policies based on our requirements. (Refer to notebook [05-custom-policy](./05-custom-policy.ipynb) for more information.)

In [ ]:
op = func.output_policy_type
op

In [ ]:
# See the implementation of the policy
print(op.policy_code)

In [ ]:
# Policies provided by Syft are available before approving the code,
# Custom policies are only safe to use once the code is approved.

assert func.output_policy is not None
assert func.input_policy is not None

func.output_policy

### Execute the Data Scientist's code

While Syft makes sure that the function is not tampered with, it does not perform any validation on the implementation itself.

**It is the Data Owner's responsibility to review the code & verify if it's safe to execute.**

In [ ]:
# Let's grab the actual executable function that was submitted by the user
users_function = func.run

If the code looks safe, we can go ahead and execute it on the private dataset

In [ ]:
mock_result = users_function(trade_data=mock_data)
mock_result

In [ ]:
real_result = users_function(trade_data=pvt_data)
real_result

### Approving a request

By calling `request.approve()`, the data scientist can execute their function on the real data, and obtain the result

In [ ]:
# Uploaded wrong result - we shared mock_result instead of the real_result
result = request.approve()
result

In [ ]:
assert isinstance(result, sy.SyftSuccess)

### Denying a request

At times you would want to deny a request in cases where the output is violating privacy, or if either of the policy is too lineant, or perhaps the code is confusing!

In [ ]:
# Deny the request with an appropriate reason
result = request.deny(
    reason=(
        "The Submitted UserCode does not add differential privacy to the output."
        "Kindly add differential privacy and resubmit the code."
    )
)
result

In [ ]:
assert isinstance(result, sy.SyftSuccess)

In [ ]:
# We can verify the status by checking our request list
project.requests

### Re-approving requests

Let's re-approve the request so that we can work with the results in the later notebooks.

In [ ]:
result = request.approve()
result

In [ ]:
# Verify the request status again
project.requests

In [ ]:
# Cleanup local datasite server

if server.server_type.value == "python":
    server.land()

Now that the code request has been approved, let's go through the [03-data-scientist-download-result](./03-data-scientist-download-result.ipynb) notebook to see how a Data Scientist can access the results.